In [1]:
import asyncio
import sounddevice as sd
import numpy as np

# AUDIO_RECORDING = 
RATE=44100
CHUNK=1024
CHANNELS=1

async def record_audio(queue):
    """Records audio and puts it into a queue."""
    with sd.InputStream(samplerate=RATE, channels=CHANNELS, dtype='float32') as stream:
        while True:
            data, overflowed = stream.read(CHUNK)
            if overflowed:
                print('Buffer overflow!')
            await queue.put(data)

async def process_audio(queue):
    """Processes audio data from the queue."""
    while True:
        data = await queue.get()
        # Process the audio data here
        print("Processing audio data:", data.shape)

# async def play_audio(queue):
#     """Plays audio data from the queue."""
#     while True:
#         data = await queue.get()
#         sd.play(data, samplerate=RATE)
#         await asyncio.sleep(0.1)  # Sleep for a short time to avoid busy-waiting

async def main():
    queue = asyncio.Queue()
    record_task = asyncio.create_task(record_audio(queue))
    process_task = asyncio.create_task(process_audio(queue))
    # play_task = asyncio.create_task(play_audio(queue))
    await asyncio.gather(record_task, process_task)

    data = await queue.get()
    sd.play(data)

if __name__ == '__main__':
    asyncio.run(main())

OSError: cannot load library 'libportaudio.so.2': libportaudio.so.2: cannot open shared object file: No such file or directory